In [1]:
!pip install tensorflow==1.15
!pip install stable-baselines[mpi]==2.10.2

In [1]:
import os
import gym
import numpy as np
import pandas as pd
from gym import spaces

/home/amirmahdi/anaconda3/envs/RL_Project2/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


There are 6 possible log levels. Here it is a list of them starting from high verbosity to low:
* 1. Trace : detail info for tracing
* 2. Debug
* 3. Info : important but normal
* 4. Warn
* 5. Error
* 6. Fatal

Among all these 6 log levels, we have decided to eliminate **Error** and **Fatal** as these two are mostly related to runtime behavioir while the information that our model try to make its logging decisions only comes from static source code features. There fore it does not have enough knowledge to decide about logs with Error and Fatal level.


**Action space prespective:**


<img src= "images/action_space_implementations.png" alt="alt text"  />

For the first run of the implementation we will consider all of the for levels (**Combined**), and after this we implement the other way to compare them together.

For the first run implementatoin the **action space** takes **6** discrete values; 
* **"0"**: not_Log
* **"1"**: Trace
* **"2"**: Debug
* **"3"**: Info
* **"4"**: Warn
* **"5"**: IDK*

**Observatio space prespective:**

Observation space is going to be a vector (box) representing each function (rows of our dataset). It has shape of (1,14) it is a vector and we have 14 features to illusterate the fuction under study.

**Reward Function:**

<img src="images/reward_function.drawio.png" />

<img src="images/sigmoid1.png" 
 longdesc="https://www.wolframalpha.com/input?i=plot+%28%28sigmoid+function+0.3x+%29+minus+0.5+%29+multiply+by+2++from+-10+to+10" title="from WolframAlpha"/>

<img src="images/sigmoid2.png" />


**Log Level:**

<img src="images/which-log-level.png" title="from which log level paper"/>

<img src="images/which-log-level1.png" title="from which log level paper"/>


In [2]:
class source_code_env(gym.Env):
    metadata = {'render.modes': ['console']}
    
    number_of_actions = 6
    
    
    actions = {
        "NOT_LOG": 0,
        "TRACE": 1,
        "DEBUG": 2,
        "INFO": 3,
        "WARN": 4,
        "IDK": 5
    }
    
    features_for_reward_func = {
        "number_of_loops": 0,
        "nesting_level": 1,
        "input_dependant_level": 2,
        "number_of_defined_threads": 3,
        "number_of_started_threads": 4,
        "number_of_join_threads": 5,
        "number_of_defined_locks": 6,
        "number_of_acquired_locks_threads": 7,
        "number_of_released_locks": 8,
        "number_of_usage_of_extra": 9,
        "number_of_usage_of_values_list": 10,
        "number_of_usage_of_select_related": 11
    }
    
    
    def __init__(self, dataset):
        super(source_code_env, self).__init__()
        self.dataset = dataset.copy()
        self.action_space = spaces.Discrete(number_of_actions)
        self.observation_space = spaces.Box(low= dataset.min().min(), 
                                            high= dataset.max().max(), 
                                            shape=(len(dataset.columns),))
        self.t = 0
        self.t_limit = len(dataset)
        
    def _action_determiner(self, obs):
        """
        The implementation logic of assiging each performance bug to a certain log level
        is based on the knowledge from previous studies mentioned above, the how common and important they are and
        also our features capabalities.
        
        arguments:
        
        obs: is supposed to be a vector (a row of the pandas dataframe);
        
        """
        synchronization_features = obs[features_for_reward_func["number_of_defined_threads"]:
            features_for_reward_func["number_of_usage_of_extra"]]
        api_related_features = obs[features_for_reward_func["number_of_usage_of_extra"]:]
        determined_action = actions["IDK"] # Default action
        
        if (obs == 0).all(): # Checking if any of features has occured
            determined_action = actions["NOT_LOG"]
            return determined_action
        elif (synchronization_features > 0).any():
            determined_action = actions["TRACE"]
            
            obs_num_threads = obs[features_for_reward_func["number_of_defined_threads"]]
            obs_num_locks = obs[features_for_reward_func["number_of_defined_locks"]]
            obs_num_join = obs[features_for_reward_func["number_of_join_threads"]]
            race_condition_metric = obs_num_threads - obs_num_locks
            deadlock_metric = obs_num_threads - obs_num_join
            if race_condition_metric > 0 or deadlock_metric > 0: 
                determined_action = actions["DEBUG"]
            return determined_action
        else:
            nested_loop = obs[features_for_reward_func["nesting_level"]] > 0
            input_dependant = obs[features_for_reward_func["input_dependant_level"]] > 0
            if nested_loop or input_dependant:
                determined_action = actions["INFO"]
                return determined_action
            elif (api_related_features != 0).any():
                determined_action = actions["WARN"]
                return determined_action
        return determined_action
        
    
    def _sigmoid(self, x_coefficient, x):
        
        """
        arguments:
        
        x_coefficient: this is to control the slope of the Sigmoid function.
        x: input of the Sigmoid function.
        """
            
        x = x_coefficient * x
        return ((1/(1 + np.exp(-x))) - 0.5) * 2
    
    def _reward(self, obs, taken_action):
        
        """
        arguments:
        
        obs: is supposed to be a vector (a row of the pandas dataframe);
        taken action: is the action taken by model based on obs. Its value could be {0, 1 ... 4}.
        """
        """
        action_based_on_obs = None
        
        if(obs.sum() == 0):
            action_based_on_obs = actions("NOT_LOG")
        else:
            action_based_on_obs = actions("TRACE")"""
        
        determined_action = _action_determiner(obs)
        
        if taken_action == determined_action: # if the answer is correct
            reward = 1 + _sigmoid()
            return reward
        elif determined_action == 0 and taken_action != 0: # if it was not supposed to be logged but it logged
            reward = -4 #Worse scenario 0 - 4
            return reward
        elif determined_action != 0 and taken_action == 0: # if it was supposed to be logged but it did not log
            reward = -4 #Worse scenario 0 - 4
            return reward
        else: # wrong log level
            reward = -abs(taken_action - determined_action) # will be -3 at the worse
            return reward
        
    
    def reset(self):
        self.t = 0
        obs = self.dataset.iloc[self.t].astype(np.float32)
        return obs
    
    def step(self, action):
        
        done = False
        
        
        if self.t >= self.t_limit:
            done = True
        
        obs = self.dataset.iloc[self.t].astype(np.float32) # getting current observation to compute reward
        reward = self._reward(obs, action)
        self.t += 1
        obs = self.dataset.iloc[self.t].astype(np.float32) # getting updated observation after taken action
        
        info={}
        
        return obs, reward, done, info

# Future Work:

* Ading Function names
* Adding file names 
* TF-IDF
* Read/Write
* API MIsuses scenarios
* GP
* Trying different implementaton of reward function
    * Adding "NO_ACTION" as one of the actions
    * Mathematical implementation
    * Negative Reward
    * Normalization of the data
* Adding more source code
* Which Log level paper:

<img src="images/which-log-level-f.png" />

<img src="images/which-log-level-f1.png" />

* IDK 

# Test

In [ ]:
class LogEnvironment(gym.Env):
    metadata = {'render.modes': ['console']}
    
    
    def __init__(self,dataset, high = None, low = None):
        super(LogEnvironment, self).__init__()
        self.rewards = []
        self.dataset = dataset
        self.states = np.array(dataset.iloc[:,:-1])
        self.state = 0
        self.predict_classes = []
        
        n_actions = 4
        self.action_space = spaces.Discrete(n_actions)
        if high is None:
            low = np.array(dataset.iloc[:,:-1].min()).astype(np.float32)
            high = np.array(dataset.iloc[:,:-1].max()).astype(np.float32)
            self.observation_space = spaces.Box(low=low, high=high, dtype=np.float32)
        else:
            self.observation_space = spaces.Box(low=low, high=high, dtype=np.float32)
        
    
    def reset(self):
        self.state = 0
        LogEnvironment.predicts.append(self.predict_classes)
        self.predict_classes = []
        return self.states[self.state].astype(np.float32)
    
    def step(self, action):
        if action == self.NoLog:
            self.predict_classes.append(0)
        elif action == self.Log:
            self.predict_classes.append(1)
        else:
            raise ValueError("Received invalid action={} which is not part of the action space".format(action))
        
        done = bool(self.state >= len(self.classes)-1)
        
        
        
        reward = 1 if self.predict_classes[self.state] == self.classes[self.state] else 0
        self.rewards.append(reward)
        
        self.state += 0 if done else 1
        
        info = {}
        
        return self.states[self.state].astype(np.float32), reward, done, info
        

In [37]:
class test:
    d = []
    p = 1
    def __init__(self, dataset):
        self.dataset = dataset.copy()
        test.d.append(p)
        
    def fun(self):
        self.dataset.append("a")

In [38]:
lst = ["Amir"]
cls = test(lst)

NameError: name 'p' is not defined

In [36]:
lst

['Amir']

In [35]:
cls.dataset

['Amir', 'a']

In [33]:
cls.d

['b']

In [6]:
features_for_reward_func = {
        "number_of_loops": 0,
        "nesting_level": 1,
        "input_dependant_level": 2,
        "number_of_defined_threads": 3,
        "number_of_started_threads": 4,
        "number_of_join_threads": 5,
        "number_of_defined_locks": 6,
        "number_of_acquired_locks_threads": 7,
        "number_of_released_locks": 8,
        "number_of_usage_of_extra": 9,
        "number_of_usage_of_values_list": 10,
        "number_of_usage_of_select_related": 11
    }
features_for_reward_func["number_of_loops"]

0

In [3]:
a = 1 > 0
print(a)

True


In [7]:
def _sigmoid(x_coefficient, x):
        
        """
        arguments:
        
        x_coefficient: this is to control the slope of the Sigmoid function.
        x: input of the Sigmoid function.
        """
            
        x = x_coefficient * x
        return ((1/(1 + np.exp(-x))) - 0.5) * 2
    
_sigmoid(0.35, 6)

0.7818063576087741

In [24]:
def _action_determiner(obs):
        """
        The implementation logic of assiging each performance bug to a certain log level
        is based on the knowledge from previous studies mentioned above, the how common and important they are and
        also our features capabalities.
        
        arguments:
        
        obs: is supposed to be a vector (a row of the pandas dataframe);
        
        """
        synchronization_features = obs[features_for_reward_func["number_of_defined_threads"]:
            features_for_reward_func["number_of_usage_of_extra"]]
        api_related_features = obs[features_for_reward_func["number_of_usage_of_extra"]:]
        determined_action = actions["IDK"] # Default action
        
        if (obs == 0).all(): # Checking if any of features has occured
            determined_action = actions["NOT_LOG"]
            return determined_action
        elif (synchronization_features > 0).any():
            determined_action = actions["TRACE"]
            
            obs_num_threads = obs[features_for_reward_func["number_of_defined_threads"]]
            obs_num_locks = obs[features_for_reward_func["number_of_defined_locks"]]
            obs_num_join = obs[features_for_reward_func["number_of_join_threads"]]
            race_condition_metric = obs_num_threads - obs_num_locks
            deadlock_metric = obs_num_threads - obs_num_join
            if race_condition_metric > 0 or deadlock_metric > 0: 
                determined_action = actions["DEBUG"]
            return determined_action
        else:
            nested_loop = obs[features_for_reward_func["nesting_level"]] > 0
            input_dependant = obs[features_for_reward_func["input_dependant_level"]] > 0
            if nested_loop or input_dependant:
                determined_action = actions["INFO"]
                return determined_action
            elif (api_related_features != 0).any():
                determined_action = actions["WARN"]
                return determined_action
            else:
                determined_action = 5
        return determined_action

In [14]:
actions = {
    "NOT_LOG": 0,
    "TRACE": 1,
    "DEBUG": 2,
    "INFO": 3,
    "WARN": 4,
    "IDK": 5
}

features_for_reward_func = {
    "number_of_loops": 0,
    "nesting_level": 1,
    "input_dependant_level": 2,
    "number_of_defined_threads": 3,
    "number_of_started_threads": 4,
    "number_of_join_threads": 5,
    "number_of_defined_locks": 6,
    "number_of_acquired_locks_threads": 7,
    "number_of_released_locks": 8,
    "number_of_usage_of_extra": 9,
    "number_of_usage_of_values_list": 10,
    "number_of_usage_of_select_related": 11
}
df = pd.read_csv("data/dataframe.csv").set_index("ID")


number_of_loops                      0.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: ee2c971f20132061d7e749267b166730ff734a7c, dtype: float32


In [25]:
for i in range(len(df)):
    obs = df.iloc[i].astype(np.float32)
    
    if _action_determiner(obs) != 0:
        print(obs)
        print("ACTION: ", _action_determiner(obs))
        print("------------------------------------------")

number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: dd15cd5b22997fef1b6efa5ffa71d6ce84de30c5, dtype: float32
ACTION:  5
------------------------------------------
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_ext

ACTION:  5
------------------------------------------
number_of_loops                      1.0
nested_loop_level                    2.0
loop_input_dependent_level           2.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: 0a25ba5991316bdda4a9b3abcee2106016df28a0, dtype: float32
ACTION:  3
------------------------------------------
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of

ACTION:  3
------------------------------------------
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: 97418507bc2fc7d55435bc3ca888b6b73c00df6e, dtype: float32
ACTION:  5
------------------------------------------
number_of_loops                      2.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of

ACTION:  3
------------------------------------------
number_of_loops                      2.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: 1925f793093581c47a5c6b2e414506b1cbae280d, dtype: float32
ACTION:  5
------------------------------------------
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of

In [26]:
for i in range(len(df)):
    print(df.iloc[i] if (df.iloc[i] != 0).any() else "-------------------")

-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: dd15cd5b22997fef1b6efa5ffa71d6ce84de30c5, dtype: float64
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
-------------------
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
nu

-------------------
-------------------
number_of_loops                      2.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: e540cdd1328b2b21e29a95405c301b9313b7c346, dtype: float64
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra            

number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage_of_values_list       0.0
number_of_usage_of_select_related    0.0
Name: 1925f793093581c47a5c6b2e414506b1cbae280d, dtype: float64
-------------------
number_of_loops                      1.0
nested_loop_level                    0.0
loop_input_dependent_level           0.0
number_of_defined_threads            0.0
number_of_started_threads            0.0
number_of_join_threads               0.0
number_of_defined_locks              0.0
number_of_acquired_locks_threads     0.0
number_of_released_locks             0.0
number_of_usage_of_extra             0.0
number_of_usage